#  Neural Networks: Regression on House Pricing Dataset
We consider a reduced version of a dataset containing house sale prices for King County, which includes Seattle. It includes homes sold between May 2014 and May 2015.

https://www.kaggle.com/harlfoxem/housesalesprediction

For each house we know 18 house features (e.g., number of bedrooms, number of bathrooms, etc.) plus its price, that is what we would like to predict.

## Insert your ID number ("numero di matricola") below

In [23]:
#put here your ``numero di matricola''
numero_di_matricola = 2122187

In [24]:
#import all packages needed
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Load the data, remove data samples/points with missing values (NaN) and take a look at them.

In [25]:
#load the data
df = pd.read_csv('kc_house_data.csv', sep = ',')

#remove the data samples with missing values (NaN)
df = df.dropna() 

df.describe()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,3.164000e+03,3.164000e+03,3164.000000,3164.000000,3164.000000,3.164000e+03,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000
mean,4.645240e+09,5.354358e+05,3.381163,2.071903,2070.027813,1.525054e+04,1.434893,0.009798,0.244311,3.459229,7.615676,1761.252212,308.775601,1967.489254,94.668774,98077.125158,47.557868,-122.212337,1982.544564,13176.302465
std,2.854203e+09,3.809004e+05,0.895472,0.768212,920.251879,4.254457e+04,0.507792,0.098513,0.776298,0.682592,1.166324,815.934864,458.977904,28.095275,424.439427,54.172937,0.140789,0.139577,686.256670,25413.180755
min,1.000102e+06,7.500000e+04,0.000000,0.000000,380.000000,6.490000e+02,1.000000,0.000000,0.000000,1.000000,3.000000,380.000000,0.000000,1900.000000,0.000000,98001.000000,47.177500,-122.514000,620.000000,660.000000
25%,2.199775e+09,3.150000e+05,3.000000,1.500000,1430.000000,5.453750e+03,1.000000,0.000000,0.000000,3.000000,7.000000,1190.000000,0.000000,1950.000000,0.000000,98032.000000,47.459575,-122.324250,1480.000000,5429.500000
50%,4.027701e+09,4.450000e+05,3.000000,2.000000,1910.000000,8.000000e+03,1.000000,0.000000,0.000000,3.000000,7.000000,1545.000000,0.000000,1969.000000,0.000000,98059.000000,47.572500,-122.226000,1830.000000,7873.000000
75%,7.358175e+09,6.402500e+05,4.000000,2.500000,2500.000000,1.122250e+04,2.000000,0.000000,0.000000,4.000000,8.000000,2150.000000,600.000000,1990.000000,0.000000,98117.000000,47.680250,-122.124000,2360.000000,10408.250000
max,9.839301e+09,5.350000e+06,8.000000,6.000000,8010.000000,1.651359e+06,3.500000,1.000000,4.000000,5.000000,12.000000,6720.000000,2620.000000,2015.000000,2015.000000,98199.000000,47.777600,-121.315000,5790.000000,425581.000000


Extract input and output data. We want to predict the price by using features other than id as input.

In [26]:
Data = df.values
# m = number of input samples
m = Data.shape[0]
print("Amount of data:",m)
Y = Data[:m,2]
X = Data[:m,3:]

Amount of data: 3164


## Data Pre-Processing

We split the data into 3 parts: one will be used for training and choosing the parameters, one for choosing among different models, and one for testing. The part for training and choosing the parameters will consist of $2/3$ of all samples, the one for choosing among different models will consist of $1/6$ of all samples, while the other part consists of the remaining $1/6$-th of all samples.

In [27]:
# Split data into train (2/3 of samples), validation (1/6 of samples), and test data (the rest)
m_train = int(2./3.*m)
m_val = int((m-m_train)/2.)
m_test = m - m_train - m_val
print("Amount of data for training and deciding parameters:",m_train)
print("Amount of data for validation (choosing among different models):",m_val)
print("Amount of data for test:",m_test)
from sklearn.model_selection import train_test_split

Xtrain_and_val, Xtest, Ytrain_and_val, Ytest = train_test_split(X, Y, test_size=m_test/m, random_state=numero_di_matricola)
Xtrain, Xval, Ytrain, Yval = train_test_split(Xtrain_and_val, Ytrain_and_val, test_size=m_val/(m_train+m_val), random_state=numero_di_matricola)

Amount of data for training and deciding parameters: 2109
Amount of data for validation (choosing among different models): 527
Amount of data for test: 528


Let's standardize the data.

In [28]:
# Data pre-processing
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(Xtrain)
Xtrain_scaled = scaler.transform(Xtrain)
Xtrain_and_val_scaled = scaler.transform(Xtrain_and_val)
Xval_scaled = scaler.transform(Xval)
Xtest_scaled = scaler.transform(Xtest)

## Neural Networks
Let's start by learning a simple neural network with 1 hidden node.
Note: we are going to use the input parameter solver='lbfgs' and random_state=numero_di_matricola to fix the random seed (so results are reproducible).

In [29]:
#let's load the MLPRegressor
from sklearn.neural_network import MLPRegressor

#let's define the model
model = MLPRegressor(hidden_layer_sizes=[1], solver="lbfgs", random_state=numero_di_matricola)

#let's learn the model on training data
model.fit(Xtrain_scaled, Ytrain)

#let's print the error (1 - R^2) on training data
print("1 - coeff. det. on training data: ", 1-model.score(Xtrain_scaled, Ytrain))

#let's print the error (1 - R^2) on validation data
print("1 - coeff. det. on validation data: ", 1-model.score(Xval_scaled, Yval))

#let's print the coefficients of the model for the input nodes (but not the bias)
print(model.coefs_)

#let's print the coefficient for the bias (i.e., the bias)
print(model.intercepts_)

1 - coeff. det. on training data:  0.2642335824632944
1 - coeff. det. on validation data:  0.3563632768099797
[array([[ -93.94633333],
       [  82.56949301],
       [ 242.27861531],
       [  38.97307496],
       [ -32.87017526],
       [ 266.7780559 ],
       [ 151.40486771],
       [  45.67793497],
       [ 293.71410636],
       [ 232.49916175],
       [  65.26772366],
       [-217.62330759],
       [  13.43727317],
       [ -80.76286646],
       [ 237.25671826],
       [ -84.07795041],
       [  75.31661837],
       [ -44.28986504]]), array([[347.85954594]])]
[array([1549.40054966]), array([-1122.7045175])]


/home/zaptop/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


## Neural Networks vs Linear Models

Let's learn a linear model on the other same data and compare the results with the simple NN above.

In [30]:
from sklearn import linear_model

l_model = linear_model.LinearRegression()
l_model.fit(Xtrain_scaled, Ytrain)

#let's print the error (1 - R^2) on training data
print("1 - coeff. det. on training data: ", 1-l_model.score(Xtrain_scaled, Ytrain))

#let's print the error (1 - R^2) on validation data
print("1 - coeff. det. on validation data: ", 1-l_model.score(Xval_scaled, Yval))

#let's print the coefficients of the model for the input nodes (but not the bias)
print(l_model.coef_)

#let's print the coefficient for the bias (i.e., the bias)
print(l_model.intercept_)

1 - coeff. det. on training data:  0.2652302356407814
1 - coeff. det. on validation data:  0.36026198851776625
[-33531.23865125  27360.94401696  85159.16056958  13311.7580626
 -10790.50315299  92835.25414918  52951.94601609  16402.93997602
  99096.96741412  81527.99857188  23271.6515263  -74435.63107285
   4948.09847017 -27784.27530216  81153.00084187 -27853.06000721
  26236.21428409 -15526.78251638]
538937.7927927783


Is there a way to make a NN network learn a linear model?

Let's first check what is the loss used by MLPRegressor...

In [31]:
#let's write the code to learn a linear model with NN: how? 

nn_l_model = MLPRegressor(hidden_layer_sizes=[1], activation="identity", solver="lbfgs", random_state=numero_di_matricola)

nn_l_model.fit(Xtrain_scaled, Ytrain)

#let's print the error (1 - R^2) on training data
print("1 - coeff. det. on training data: ", 1-nn_l_model.score(Xtrain_scaled, Ytrain))

#let's print the error (1 - R^2) on validation data
print("1 - coeff. det. on validation data: ", 1-nn_l_model.score(Xval_scaled, Yval))

#let's print the coefficients of the model for the input nodes (but not the bias)
print(nn_l_model.coefs_)

#let's print the coefficient for the bias (i.e., the bias)
print(nn_l_model.intercepts_)

print("Estimate coefs of the linear model: ", nn_l_model.coefs_[0]*nn_l_model.coefs_[1][0])
print("Estimate intercept of the linear model: ", nn_l_model.intercepts_[0]*nn_l_model.coefs_[1][0] + nn_l_model.intercepts_[1])



1 - coeff. det. on training data:  0.2984328753045362
1 - coeff. det. on validation data:  0.3700431327721536
[array([[  -8481.87260918],
       [ -55230.08742147],
       [-106511.44961777],
       [  -2116.74048647],
       [  -8463.80443313],
       [-114670.68425838],
       [-106665.95830303],
       [ -37283.31651882],
       [ -99662.62556233],
       [ -79219.55678163],
       [ -69983.89161651],
       [  62797.87032661],
       [ -33911.5716448 ],
       [ -22490.53028403],
       [ -99034.19466626],
       [  43897.85471387],
       [ -84986.84929804],
       [    276.23689347]]), array([[-0.64147606]])]
[array([-516967.62471555]), array([207317.10280274])]
Estimate coefs of the linear model:  [[  5440.91822702]
 [ 35428.77890036]
 [ 68324.54509928]
 [  1357.83834837]
 [  5429.32792463]
 [ 73558.49879326]
 [ 68423.65872202]
 [ 23916.35500298]
 [ 63931.18842512]
 [ 50817.44919908]
 [ 44892.99109283]
 [-40283.3304651 ]
 [ 21753.46138418]
 [ 14427.13676522]
 [ 63528.06504961]
 

Note that there is an $\ell_2$ regularization term in MLPRegressor. What about making it smaller?

In [32]:
#let's write the code to learn a linear model with NN: how? 

nn_l_model = MLPRegressor(hidden_layer_sizes=[1], activation="identity", alpha=1e-20, solver="lbfgs", random_state=numero_di_matricola)

nn_l_model.fit(Xtrain_scaled, Ytrain)

#let's print the error (1 - R^2) on training data
print("1 - coeff. det. on training data: ", 1-nn_l_model.score(Xtrain_scaled, Ytrain))

#let's print the error (1 - R^2) on validation data
print("1 - coeff. det. on validation data: ", 1-nn_l_model.score(Xval_scaled, Yval))

#let's print the coefficients of the model for the input nodes (but not the bias)
print(nn_l_model.coefs_)

#let's print the coefficient for the bias (i.e., the bias)
print(nn_l_model.intercepts_)

print("Estimate coefs of the linear model: ", nn_l_model.coefs_[0]*nn_l_model.coefs_[1][0])
print("Estimate intercept of the linear model: ", nn_l_model.intercepts_[0]*nn_l_model.coefs_[1][0] + nn_l_model.intercepts_[1])



1 - coeff. det. on training data:  0.2984320037355004
1 - coeff. det. on validation data:  0.37004217736260436
[array([[  -8481.35765725],
       [ -55229.4813472 ],
       [-106510.41358662],
       [  -2116.94061212],
       [  -8463.98120712],
       [-114669.43262941],
       [-106664.25419832],
       [ -37282.60547453],
       [ -99661.98577074],
       [ -79219.30270512],
       [ -69982.28286273],
       [  62796.7449222 ],
       [ -33910.79626467],
       [ -22489.27293232],
       [ -99033.05243232],
       [  43896.60964058],
       [ -84986.14610492],
       [    276.04569676]]), array([[-0.64148327]])]
[array([-516963.74260949]), array([207315.90716832])]
Estimate coefs of the linear model:  [[  5440.64904104]
 [ 35428.78827561]
 [ 68324.6483592 ]
 [  1357.98198551]
 [  5429.50233899]
 [ 73558.5225719 ]
 [ 68423.3345378 ]
 [ 23916.16766083]
 [ 63931.49649192]
 [ 50817.85731858]
 [ 44892.46362827]
 [-40283.061256  ]
 [ 21753.20846426]
 [ 14426.49233265]
 [ 63528.04627759]


## More Complex NNs

Let's try more complex NN, for example increasing the number of nodes in the only hidden layer, or increasing the number of hidden layers.

Let's build a NN with 2 nodes in the only hidden layer

In [33]:
#let's build a NN with 2 nodes in the only hidden layer

nn_l_model = MLPRegressor(hidden_layer_sizes=[2], solver="lbfgs", random_state=numero_di_matricola)

nn_l_model.fit(Xtrain_scaled, Ytrain)

#let's print the error (1 - R^2) on training data
print("1 - coeff. det. on training data: ", 1-nn_l_model.score(Xtrain_scaled, Ytrain))

#let's print the error (1 - R^2) on validation data
print("1 - coeff. det. on validation data: ", 1-nn_l_model.score(Xval_scaled, Yval))

#let's print the coefficients of the model for the input nodes (but not the bias)
print(nn_l_model.coefs_)

#let's print the coefficient for the bias (i.e., the bias)
print(nn_l_model.intercepts_)

1 - coeff. det. on training data:  0.1937450307041353
1 - coeff. det. on validation data:  0.47469253155802316
[array([[ -14.1916097 ,   16.11330779],
       [  15.02424839,  141.42922994],
       [  36.14372244,   50.84274681],
       [   8.75312303, -140.01942305],
       [  -0.76210372,  -94.92845952],
       [  64.51071828, -369.40102704],
       [  13.88947065,   68.2819302 ],
       [   9.00747001,   66.66535304],
       [  52.56650028,   97.59751024],
       [  33.14195677,   87.44630027],
       [  13.90777777,  -57.68593994],
       [ -33.42355658, -254.68190238],
       [  -0.62396503,   18.71452374],
       [  -7.54112452, -413.10357014],
       [  44.94940431,  114.7277073 ],
       [ -10.05097091, -504.03851175],
       [  14.2338418 ,  215.63551256],
       [  -5.8182603 ,  -28.49163541]]), array([[1753.29383267],
       [1589.43820249]])]
[array([  296.03664913, -1272.70655256]), array([4465.32216347])]


/home/zaptop/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Let's build a NN with 5 nodes in the only hidden layer

In [34]:
#let's build a NN with 5 nodes in the only hidden layer

nn_l_model = MLPRegressor(hidden_layer_sizes=[5], solver="lbfgs", random_state=numero_di_matricola, max_iter=2000)

nn_l_model.fit(Xtrain_scaled, Ytrain)

#let's print the error (1 - R^2) on training data
print("1 - coeff. det. on training data: ", 1-nn_l_model.score(Xtrain_scaled, Ytrain))

#let's print the error (1 - R^2) on validation data
print("1 - coeff. det. on validation data: ", 1-nn_l_model.score(Xval_scaled, Yval))

#let's print the coefficients of the model for the input nodes (but not the bias)
print(nn_l_model.coefs_)

#let's print the coefficient for the bias (i.e., the bias)
print(nn_l_model.intercepts_)

1 - coeff. det. on training data:  0.1116397394187949
1 - coeff. det. on validation data:  0.2467343033837931
[array([[-7.82603575e+01,  1.02345051e+01,  5.73524092e+01,
         3.37441205e+01,  8.09406082e+01],
       [ 9.26224114e+00,  3.52474191e+01,  3.18033719e+01,
        -2.55799949e+01,  1.28590009e+01],
       [ 2.06125200e+02,  1.57391561e+02,  1.38533080e+01,
         1.87508540e+01, -9.87453222e+01],
       [-1.69737073e+02,  3.47354762e+02,  3.11632805e+02,
        -1.05616002e+02,  3.40766436e+01],
       [-1.18322559e+01, -1.23002642e+02, -4.95876262e+01,
        -4.57209746e+01,  1.95498486e+01],
       [ 2.05628955e+01,  1.31710595e+02, -3.77969111e+01,
        -2.80250635e+01, -1.16352297e+02],
       [ 2.32938897e+02, -1.61781747e+02, -2.13789474e+02,
        -1.09376362e+02, -1.93267617e+02],
       [-3.28964246e+01,  1.03136340e+02,  9.14117603e+01,
        -1.66822056e+01,  3.19024363e+01],
       [ 2.72301597e+02,  1.18816534e+02, -5.97884840e-01,
        -5.640

Note that with a smaller number of iterations we had a larger error on training set but a smaller error on validation data -> "early stopping is a form of regularization"

Let's build a NN with 10 nodes in the only hidden layer

In [35]:
#let's build a NN with 10 nodes in the only hidden layer

nn_l_model = MLPRegressor(hidden_layer_sizes=[10], solver="lbfgs", random_state=numero_di_matricola, max_iter=2000)

nn_l_model.fit(Xtrain_scaled, Ytrain)

#let's print the error (1 - R^2) on training data
print("1 - coeff. det. on training data: ", 1-nn_l_model.score(Xtrain_scaled, Ytrain))

#let's print the error (1 - R^2) on validation data
print("1 - coeff. det. on validation data: ", 1-nn_l_model.score(Xval_scaled, Yval))

#let's print the coefficients of the model for the input nodes (but not the bias)
print(nn_l_model.coefs_)

#let's print the coefficient for the bias (i.e., the bias)
print(nn_l_model.intercepts_)

1 - coeff. det. on training data:  0.06944405861911918
1 - coeff. det. on validation data:  0.2601037704883664
[array([[-2.42644942e+01,  1.79514002e+01, -2.95017866e+00,
         5.97517504e+00,  4.81273335e+00, -3.98193938e+00,
         1.06267598e+01,  6.15436549e+01,  7.95211919e+01,
        -4.15880436e+01],
       [-2.74537590e+02, -5.69372466e+01, -1.08483124e+02,
         2.34916693e+02,  1.58681828e+02, -2.62142558e+01,
         2.53558251e+02, -3.75536379e+02, -3.13284841e+02,
         1.78896022e+02],
       [ 3.84935730e+02,  4.83662722e+01,  1.72218454e+02,
        -1.70480419e+02,  6.64477712e+01,  7.21541870e+00,
        -2.63469533e+02,  1.00200961e+02,  2.58244725e+00,
         1.93230942e+01],
       [-9.13078096e+01, -7.24569501e+00,  6.05202829e+01,
        -6.87430228e+01, -4.98377513e+01, -5.67201586e+01,
        -1.19557280e+02, -4.11845975e+02,  8.04528597e+00,
         1.18418173e+01],
       [-1.19985425e+02,  2.82325261e+01, -5.98018441e+01,
         3.392124

/home/zaptop/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Let's build a NN with 100 nodes in the only hidden layer. Note that this is the default!

In [36]:
#let's build a NN with 2 nodes in the only hidden layer

nn_l_model = MLPRegressor(hidden_layer_sizes=[100], solver="lbfgs", random_state=numero_di_matricola, max_iter=2000)

nn_l_model.fit(Xtrain_scaled, Ytrain)

#let's print the error (1 - R^2) on training data
print("1 - coeff. det. on training data: ", 1-nn_l_model.score(Xtrain_scaled, Ytrain))

#let's print the error (1 - R^2) on validation data
print("1 - coeff. det. on validation data: ", 1-nn_l_model.score(Xval_scaled, Yval))

#let's print the coefficients of the model for the input nodes (but not the bias)
print(nn_l_model.coefs_)

#let's print the coefficient for the bias (i.e., the bias)
print(nn_l_model.intercepts_)

1 - coeff. det. on training data:  0.0062687566924787275
1 - coeff. det. on validation data:  0.4298084342865309
[array([[-164.24414388, -238.44614535,   84.10811866, ...,   59.34332708,
         149.23985394,   40.57680661],
       [ -47.46898151,  122.3617751 ,    8.94751253, ..., -270.42458219,
         -89.6434295 ,  110.43986335],
       [  15.33772719,  177.37637621,  -87.50281908, ...,   96.33582402,
        -156.50109356,   14.31348157],
       ...,
       [-101.51111947,   96.59873243,  156.93100721, ...,   -8.94088619,
         -42.99881542,  218.45162125],
       [ 260.10561122, -165.08346804,  -65.3598768 , ...,  -94.05320601,
         245.90821083,   -2.69516754],
       [-225.61697602, -131.88432558,    0.7960276 , ...,  -58.30925718,
          -3.68295759,  235.51060513]]), array([[ 273.37303662],
       [-329.43925831],
       [-396.02933698],
       [ 205.2337906 ],
       [ 424.41200997],
       [ 128.66733694],
       [-369.18218819],
       [ 532.2777961 ],
       [

/home/zaptop/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Let's try 2 layers, 1 node each

In [37]:
#let's build a NN with 2 hidden layers, 1 node each

nn_l_model = MLPRegressor(hidden_layer_sizes=[1, 1], solver="lbfgs", random_state=numero_di_matricola)

nn_l_model.fit(Xtrain_scaled, Ytrain)

#let's print the error (1 - R^2) on training data
print("1 - coeff. det. on training data: ", 1-nn_l_model.score(Xtrain_scaled, Ytrain))

#let's print the error (1 - R^2) on validation data
print("1 - coeff. det. on validation data: ", 1-nn_l_model.score(Xval_scaled, Yval))

#let's print the coefficients of the model for the input nodes (but not the bias)
print(nn_l_model.coefs_)

#let's print the coefficient for the bias (i.e., the bias)
print(nn_l_model.intercepts_)

1 - coeff. det. on training data:  1.0000000000000002
1 - coeff. det. on validation data:  1.0000443251319044
[array([[ 9230.21442357],
       [15248.93577929],
       [14346.40929769],
       [-1617.04019545],
       [13217.83480942],
       [ 7470.96851787],
       [ 8091.89825946],
       [ -809.11902251],
       [17060.8411542 ],
       [13577.46221654],
       [ 4203.00582684],
       [ 5739.52369874],
       [-7253.79767882],
       [ 1307.1768639 ],
       [12394.80893973],
       [ 4333.08969015],
       [14800.0304396 ],
       [-2429.94592455]]), array([[-2909.5938515]]), array([[48736.44870344]])]
[array([43553.18073903]), array([-89897.4347007]), array([538937.79554039])]


Let's try 2 layers, 2 nodes each

In [38]:
#let's build a NN with 2 layers, 2 nodes each

nn_l_model = MLPRegressor(hidden_layer_sizes=[2, 2], solver="lbfgs", random_state=numero_di_matricola)

nn_l_model.fit(Xtrain_scaled, Ytrain)

#let's print the error (1 - R^2) on training data
print("1 - coeff. det. on training data: ", 1-nn_l_model.score(Xtrain_scaled, Ytrain))

#let's print the error (1 - R^2) on validation data
print("1 - coeff. det. on validation data: ", 1-nn_l_model.score(Xval_scaled, Yval))

#let's print the coefficients of the model for the input nodes (but not the bias)
print(nn_l_model.coefs_)

#let's print the coefficient for the bias (i.e., the bias)
print(nn_l_model.intercepts_)

1 - coeff. det. on training data:  0.20837940392575094
1 - coeff. det. on validation data:  0.3310303860187154
[array([[ 26.56583599, -20.27234061],
       [ -2.05867649,  12.56904317],
       [-57.71173137,  50.93725048],
       [  2.75808426,  -8.22470531],
       [ 30.85678978, -16.44416277],
       [-28.02649993,  28.35706591],
       [-35.98002601,  27.36191647],
       [  5.46278314,   4.67228521],
       [ -9.54347385,  46.15577887],
       [-56.71915308,  48.81093893],
       [-11.7266297 ,  11.92179099],
       [ 27.40624343, -42.0077496 ],
       [ -9.93027777,   5.38876038],
       [ 22.70746127, -25.20142944],
       [  0.12006192,  37.2345877 ],
       [ 24.15894639, -31.06176441],
       [ 17.60129395,   9.23604917],
       [ 21.90477375, -13.96459769]]), array([[-13.54556816,  39.191397  ],
       [-14.95439671,  79.90665079]]), array([[116.33009242],
       [ 36.10497144]])]
[array([126.89583793, 103.69972217]), array([-25.89921272,  23.94827736]), array([69.03815987])]

/home/zaptop/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Let's try 2 layers, 10 nodes each

In [39]:
#let's build a NN with 2 layers, 10 nodes each

nn_l_model = MLPRegressor(hidden_layer_sizes=[10, 10], solver="lbfgs", random_state=numero_di_matricola)

nn_l_model.fit(Xtrain_scaled, Ytrain)

#let's print the error (1 - R^2) on training data
print("1 - coeff. det. on training data: ", 1-nn_l_model.score(Xtrain_scaled, Ytrain))

#let's print the error (1 - R^2) on validation data
print("1 - coeff. det. on validation data: ", 1-nn_l_model.score(Xval_scaled, Yval))

#let's print the coefficients of the model for the input nodes (but not the bias)
print(nn_l_model.coefs_)

#let's print the coefficient for the bias (i.e., the bias)
print(nn_l_model.intercepts_)

1 - coeff. det. on training data:  0.11871581269998766
1 - coeff. det. on validation data:  0.360502554579903
[array([[-2.98355367e+00,  4.63813658e+00, -5.64906787e+00,
        -4.26353084e+00,  7.75483878e-02, -1.50115477e+01,
         1.08548399e+00,  1.54026023e+01, -7.10386732e+00,
        -1.73872996e+00],
       [ 1.25887058e+01,  4.39375767e+00, -1.28205704e+01,
        -7.74438907e+00,  5.79916197e+00,  2.99259762e+00,
        -1.37317808e+01, -1.39019315e+01, -7.82599938e+00,
         2.68562934e+01],
       [-3.33158506e+01,  5.55598906e+01, -2.27040740e+00,
        -3.60058421e+01, -7.54886178e+00, -1.85652326e+01,
         1.82120550e+01, -1.04745614e+01,  1.13024768e+01,
         4.88606421e+00],
       [-8.08594926e+00,  1.00266681e+01,  2.60877375e+00,
        -6.07385822e+00, -2.81768642e-01,  4.78646478e+00,
         2.99486250e+00, -3.73098720e+00, -3.29965899e+01,
        -3.06228328e+01],
       [ 4.16450366e+01, -2.03181275e+01, -1.12355132e+01,
         1.1823726

/home/zaptop/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Let's try 2 layers, 100 nodes each

In [40]:
#let's build a NN with 2 layers, 100 nodes each

nn_l_model = MLPRegressor(hidden_layer_sizes=[100, 100], solver="lbfgs", random_state=numero_di_matricola)

nn_l_model.fit(Xtrain_scaled, Ytrain)

#let's print the error (1 - R^2) on training data
print("1 - coeff. det. on training data: ", 1-nn_l_model.score(Xtrain_scaled, Ytrain))

#let's print the error (1 - R^2) on validation data
print("1 - coeff. det. on validation data: ", 1-nn_l_model.score(Xval_scaled, Yval))

#let's print the coefficients of the model for the input nodes (but not the bias)
print(nn_l_model.coefs_)

#let's print the coefficient for the bias (i.e., the bias)
print(nn_l_model.intercepts_)

1 - coeff. det. on training data:  0.022518176142853452
1 - coeff. det. on validation data:  0.3743348717774784
[array([[ -8.75998002,   1.46165199, -19.6765628 , ...,   4.20432151,
         -6.43594857,  10.28535403],
       [ -9.29353757,  -4.25608019,   7.47143874, ..., -13.09997412,
         12.50532727, -16.11181545],
       [ -0.75790048,  -9.0972625 ,  13.20345039, ...,   3.2371463 ,
          8.37945448,  -4.41004741],
       ...,
       [-25.15743924,  -7.55408902, -18.45743691, ..., -23.08895272,
         11.96196526,  10.75974761],
       [  8.19551506,  12.97098305,  10.59815647, ...,  16.35068339,
         -5.12870432,  -7.93803335],
       [-34.98391501,   8.9153226 , -10.21327588, ...,   4.84813359,
         -0.08013858,  -0.08824526]]), array([[-0.16823828, -0.95011617, -0.6673614 , ..., -2.19414511,
        -1.68833476, -1.06038025],
       [ 0.88316636,  2.20191901, -0.62330093, ...,  4.14617051,
         2.81220096, -0.70079407],
       [ 1.07447061,  3.37596501, -0.

/home/zaptop/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


So it seems that 1 layer (and default number of iterations) works best for this dataset. Let's try 5-fold cross-validation with number of nodes in the hidden layer between 1 and 20.
Note that we use train and validation data together, since we are doing cross-validation.

In [41]:
from sklearn.model_selection import GridSearchCV

#COMPLETE

Now let's check what is the best parameter, and compare the best NNs with the linear model (learned on train and validation) on test data.

In [42]:
#let's print the best model according to grid search
#COMPLETE

#let's print the error 1-R^2 for the best model
#COMPLETE

Let compare the error of the best NN on train and validation and on test data.

In [43]:
#COMPLETE

Now let's learn the linear model on train and validation, and get error (1-R^2) on train and validation and on test data.

In [44]:
#COMPLETE

Note: MLPRegressor has several other parameters!